## Logistic Regression(sklearn)

In [8]:
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
X, y = load_iris(return_X_y=True)
clf = LogisticRegression(random_state=0).fit(X, y)
clf.predict(X[:2, :])
clf.predict_proba(X[:2, :])
clf.score(X, y)

d:\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.9733333333333334

## MaxEntropy(Self build version, IIS&~~DFP~~ algorithms)

### 出现了莫名错误不知道为啥不能把IIS算法和DFP算法合并在一个class里面......

In [186]:
import math
from collections import defaultdict
import numpy as np
from scipy.optimize import fminbound
import copy


class MaxEnt:
    def __init__(self, epsilon=1e-4, maxstep=200, algorithms='', verbose=False):
        print('this tool use a special way to get more eigenfunction')
        self.epsilon = epsilon
        self.maxstep = maxstep
        self.w = None  # 特征函数的权重，其实就是omega
        self.labels = None  # 标签
        self.fea_list = []  # 特征函数
        # 这里用了defaultdict这个方法，其实就是为字典设置了一个默认值，这样就防止了从字典里面按照key取值key不存在而出错的情况
        self.px = defaultdict(lambda: 0)  # 经验边缘分布概率
        self.pxy = defaultdict(lambda: 0)  # 经验联合分布概率
        self.exp_fea = defaultdict(lambda: 0)  # 每个特征的期望
        self.data_list = []  # 样本集，元素为tuple((X),y)
        self.N = None  # 样本总量
        self.M = None  # 某个训练样本包含特征的总数，这里假设每个样本的M值相同，即M为常数。
        self.zw = None
        self.n_fea = None  # 特征函数的总数
        self.n_ilter = None # 实际的迭代次数
        self._algorithms = algorithms # 选算法
        self._verbose = verbose

    def init_param(self, X_data, y_data):
        # 根据传入的数据集(数组)初始化模型参数
        self.N = X_data.shape[0]
        self.labels = np.unique(y_data)

        self.fea_func(X_data, y_data) # 往fea_list里面放进了需要的特征函数
        self.n_fea = len(self.fea_list)
        self.w = np.zeros(self.n_fea) # 初始化w值
        self._exp_fea(X_data, y_data) # 往exp_fea放进了计算得到的exp_fea
        return

    def fea_func(self, X_data, y_data, rules=None):
        # 这里特征函数的构造方法很巧妙
        # 特征函数
        if rules is None:  # 若没有特征提取规则，则直接构造特征，此时每个样本没有缺失值的情况下的特征个数相同，等于维度
            for X, y in zip(X_data, y_data):
                X = tuple(X)
                self.px[X] += 1.0 / self.N  # X的经验边缘分布
                self.pxy[(X, y)] += 1.0 / self.N  # X,y的经验联合分布

                for dimension, val in enumerate(X):
                    key = (dimension, val, y)
                    if not key in self.fea_list:
                        self.fea_list.append(key)  # 特征函数，由 (维度+维度下的值+标签) 构成的元组
            self.M = X_data.shape[1]
        else:
            self.M = defaultdict(int)  # 字典存储每个样本的特征总数，defaultdict会自动构造一个字典
            for i in range(self.N):
                self.M[i] = X_data.shape[1]
            pass  # 根据具体规则构建，因为这里并没有具体规则，所以就写了个pass

    def _exp_fea(self, X_data, y_data):
        # 计算特征的经验期望值
        for X, y in zip(X_data, y_data):
            for dimension, val in enumerate(X):
                fea = (dimension, val, y)
                self.exp_fea[fea] += self.pxy[(tuple(X), y)]  # 上面fea_func里面算过了，这里只是取出来，用特征函数的形式存放起来
        return

    def _py_X(self, X):
        # 当前w下的条件分布概率,输入向量X和y的条件概率
        py_X = defaultdict(float)
        
        # 这里计算py_x以及计算规范化因子的方法也很牛逼......

        for y in self.labels:
            s = 0
            for dimension, val in enumerate(X):
                tmp_fea = (dimension, val, y)
                for i in self.fea_list:
                    if tmp_fea == i:  # 输入X包含的特征
                        s += self.w[self.fea_list.index(tmp_fea)] # 因为特征函数对于存在的特征直接是1，所以这里只需要self.w就行
            py_X[y] = np.exp(s)

        normalizer = sum(py_X.values()) # 这个normalizer就是规范化因子，好牛逼的方法......
        self.zw = normalizer # 保存一下zw
        for key, val in py_X.items():
            py_X[key] = val / normalizer
        return py_X

    def _est_fea(self, X_data, y_data):
        # 基于当前模型，获取每个特征估计期望
        est_fea = defaultdict(float)
        for X, y in zip(X_data, y_data):
            py_x = self._py_X(X)[y]
            for dimension, val in enumerate(X):
                est_fea[(dimension, val, y)] += self.px[tuple(X)] * py_x
        return est_fea

    def GIS(self, X_data, y_data):
        # GIS算法更新delta
        # 通用迭代算法（这算法好像用的很少）
        # 算法具体解释参考：https://blog.csdn.net/u014688145/article/details/55003910
        # 这里的delta就是参数w每次的迭代变化值
        est_fea = self._est_fea(X_data, y_data)
        delta = np.zeros(self.n_fea)
        # 这是只写了一次迭代
        for j in range(self.n_fea):
            try:
                delta[j] = 1 / self.M * math.log(self.exp_fea[self.fea_list[j]] / est_fea[self.fea_list[j]])
            except:
                continue
        delta = delta / delta.sum()  # 归一化，防止某一个特征权重过大导致，后续计算超过范围
        # 这个感觉有点意义不明
        return delta

    def IIS(self, delta, X_data, y_data):
        # IIS算法更新delta
        # 当self.M为常数的时候，IIS/GIS两个算法完全等价，delta也存在显式解（统计学习书上有）
        g = np.zeros(self.n_fea)
        g_diff = np.zeros(self.n_fea)
        for j in range(self.n_fea):
            for k in range(self.N):
                # 这下面的公式其实是统计学习书上推导的，实际上就是牛顿法
                # 不过这个迭代公式似乎省略了一个exp_fea，但是从迭代的角度来看好像完全是不影响的
                g[j] += self.px[tuple(X_data[k])] * self._py_X(X_data[k])[y_data[k]] * math.exp(delta[j] * self.M)
                g_diff[j] += self.px[tuple(X_data[k])] * self._py_X(X_data[k])[y_data[k]] * math.exp(delta[j] * self.M) * self.M
            g[j] -= self.exp_fea[j]
            delta[j] -= g[j] / g_diff[j]
        return delta


    def fit(self, X_data, y_data):
        self.n_ilter = 0
        # 训练，迭代更新wi
        self.init_param(X_data, y_data)
        
        # 选择算法
        if self._algorithms == '':
            if isinstance(self.M, int): # 判断self.M是不是一个常数，如果有自定义的特征提取规则，那会是个字典，不是int
                i = 0
                while i < self.maxstep:
                    self.n_ilter += 1
                    i += 1
                    delta = self.GIS(X_data, y_data)
                    if max(abs(delta)) < self.epsilon:  # 所有的delta都小于阈值时，停止迭代
                        break
                    self.w += delta
            else:
                i = 0
                delta = np.random.rand(self.n_fea)
                while i < self.maxstep:
                    self.n_ilter += 1
                    i += 1
                    delta = self.IIS(delta, X_data, y_data)
                    if max(abs(delta)) < self.epsilon:
                        break
                    self.w += delta   
        elif self._algorithms == 'IIS':
            if isinstance(self.M, int):
                print('Ur dataset\'s all features are same, so GIS algorithm will be used(In this case, IIS == GIS)')
                i = 0
                while i < self.maxstep:
                    self.n_ilter += 1
                    i += 1
                    delta = self.GIS(X_data, y_data)
                    if max(abs(delta)) < self.epsilon:  # 所有的delta都小于阈值时，停止迭代
                        break
                    self.w += delta
            else:        
                i = 0
                delta = np.random.rand(self.n_fea)
                while i < self.maxstep:
                    self.n_ilter += 1
                    i += 1
                    delta = self.IIS(delta, X_data, y_data)
                    if max(abs(delta)) < self.epsilon:
                        break
                    self.w += delta
        elif self._algorithms == 'GIS':
            if isinstance(self.M, int):
                i = 0
                while i < self.maxstep:
                    self.n_ilter += 1
                    i += 1
                    delta = self.GIS(X_data, y_data)
                    if max(abs(delta)) < self.epsilon:  # 所有的delta都小于阈值时，停止迭代
                        break
                    self.w += delta
            else:
                print('Using IIS(different feature numbers among samples)')
                i = 0
                delta = np.random.rand(self.n_fea)
                while i < self.maxstep:
                    self.n_ilter += 1
                    i += 1
                    delta = self.IIS(delta, X_data, y_data)
                    if max(abs(delta)) < self.epsilon:
                        break
                    self.w += delta
        elif self._algorithms == 'DFP':
            
            print('Pleause the another module called MaxEntDFP')
                
        else:
            print('please decide algorithms in IIS&GIS&DFP, defult is GIS/IIS')
            return
        
        print('实际迭代次数{}'.format(self.n_ilter))
        
        return
    

    def predict(self, X):
        # 输入x(数组)，返回条件概率最大的标签
        py_x = self._py_X(X)
        best_label = max(py_x, key=py_x.get) # 注意这个取预测最值的方法
        if self._verbose:
            print("模型权重：{}".format(self.w))
            print("模型特征函数量：{}".format(self.n_fea))
        return best_label        
    
    
######################################################################################################################################
# DFP
######################################################################################################################################


class MaxEntDFP:
    def __init__(self, epsilon, max_iter=1000, distance=0.01):
        """
        最大熵的DFP算法
        :param epsilon: 迭代停止阈值
        :param max_iter: 最大迭代次数
        :param distance: 一维搜索的长度范围
        """
        print('this algorithms only support str data for now(23.09.19)')
        self.distance = distance
        self.epsilon = epsilon
        self.max_iter = max_iter
        self.w = None
        self._dataset_X = None
        self._dataset_y = None
        # 标签集合，相当去去重后的y
        self._y = set()
        # key为(x,y), value为对应的索引号ID
        self._xyID = {}
        # key为对应的索引号ID, value为(x,y)
        self._IDxy = {}
        # 经验分布p(x,y)
        self._pxy_dic = defaultdict(int)
        # 样本数
        self._N = 0
        # 特征键值(x,y)的个数
        self._n = 0
        # 实际迭代次数
        self.n_iter_ = 0

    # 初始化参数
    def init_params(self, X, y):
        self._dataset_X = copy.deepcopy(X)
        self._dataset_y = copy.deepcopy(y)
        self._N = X.shape[0]

        for i in range(self._N):
            xi, yi = X[i], y[i]
            self._y.add(yi)
            for _x in xi:
                self._pxy_dic[(_x, yi)] += 1

        self._n = len(self._pxy_dic)
        # 初始化权重w
        self.w = np.zeros(self._n)

        for i, xy in enumerate(self._pxy_dic):
            self._pxy_dic[xy] /= self._N
            self._xyID[xy] = i
            self._IDxy[i] = xy

    def calc_zw(self, X, w):
        """书中第100页公式6.23，计算Zw(x)"""
        zw = 0.0
        for y in self._y:
            zw += self.calc_ewf(X, y, w)
        return zw

    def calc_ewf(self, X, y, w):
        """书中第100页公式6.22，计算分子"""
        sum_wf = self.calc_wf(X, y, w)
        return np.exp(sum_wf)

    def calc_wf(self, X, y, w):
        sum_wf = 0.0
        for x in X:
            if (x, y) in self._pxy_dic:
                sum_wf += w[self._xyID[(x, y)]]
        return sum_wf

    def calc_pw_yx(self, X, y, w):
        """计算Pw(y|x)"""
        return self.calc_ewf(X, y, w) / self.calc_zw(X, w)

    def calc_f(self, w):
        """计算f(w)"""
        fw = 0.0
        for i in range(self._n):
            x, y = self._IDxy[i]
            for dataset_X in self._dataset_X:
                if x not in dataset_X:
                    continue
                fw += np.log(self.calc_zw(x, w)) - \
                    self._pxy_dic[(x, y)] * self.calc_wf(dataset_X, y, w)

        return fw

    # DFP算法
    def fit(self, X, y):
        self.init_params(X, y)

        def calc_dfw(i, w):
            """计算书中第107页的拟牛顿法f(w)的偏导"""

            def calc_ewp(i, w):
                """计算偏导左边的公式"""
                ep = 0.0
                x, y = self._IDxy[i]
                for dataset_X in self._dataset_X:
                    if x not in dataset_X:
                        continue
                    ep += self.calc_pw_yx(dataset_X, y, w) / self._N
                return ep

            def calc_ep(i):
                """计算关于经验分布P(x,y)的期望值"""
                (x, y) = self._IDxy[i]
                return self._pxy_dic[(x, y)]

            return calc_ewp(i, w) - calc_ep(i)

        # 算出g(w)，是n*1维矩阵
        def calc_gw(w):
            return np.array([[calc_dfw(i, w) for i in range(self._n)]]).T

        # （1）初始正定对称矩阵，单位矩阵
        Gk = np.array(np.eye(len(self.w), dtype=float))

        # （2）计算g(w0)
        w = self.w
        gk = calc_gw(w)
        # 判断gk的范数是否小于阈值
        if np.linalg.norm(gk, ord=2) < self.epsilon:
            self.w = w
            return

        k = 0
        for _ in range(self.max_iter):
            # （3）计算pk
            pk = -Gk.dot(gk)

            # 梯度方向的一维函数
            def _f(x):
                z = w + np.dot(x, pk).T[0]
                return self.calc_f(z)

            # （4）进行一维搜索，找到使得函数最小的lambda
            _lambda = fminbound(_f, -self.distance, self.distance)

            delta_k = _lambda * pk
            # （5）更新权重
            w += delta_k.T[0]

            # （6）计算gk+1
            gk1 = calc_gw(w)
            # 判断gk1的范数是否小于阈值
            if np.linalg.norm(gk1, ord=2) < self.epsilon:
                self.w = w
                break
            # 根据DFP算法的迭代公式（附录B.24公式）计算Gk
            yk = gk1 - gk
            Pk = delta_k.dot(delta_k.T) / (delta_k.T.dot(yk))
            Qk = Gk.dot(yk).dot(yk.T).dot(Gk) / (yk.T.dot(Gk).dot(yk)) * (-1)
            Gk = Gk + Pk + Qk
            gk = gk1

            # （7）置k=k+1
            k += 1

        self.w = w
        self.n_iter_ = k

    def predict(self, x):
        result = {}
        for y in self._y:
            prob = self.calc_pw_yx(x, y, self.w)
            result[y] = prob

        return result


# test function
def main(algorithms=''):
    if algorithms == 'DFP':
        # 训练数据集
        dataset = np.array([['no', 'sunny', 'hot', 'high', 'FALSE'],
                            ['no', 'sunny', 'hot', 'high', 'TRUE'],
                            ['yes', 'overcast', 'hot', 'high', 'FALSE'],
                            ['yes', 'rainy', 'mild', 'high', 'FALSE'],
                            ['yes', 'rainy', 'cool', 'normal', 'FALSE'],
                            ['no', 'rainy', 'cool', 'normal', 'TRUE'],
                            ['yes', 'overcast', 'cool', 'normal', 'TRUE'],
                            ['no', 'sunny', 'mild', 'high', 'FALSE'],
                            ['yes', 'sunny', 'cool', 'normal', 'FALSE'],
                            ['yes', 'rainy', 'mild', 'normal', 'FALSE'],
                            ['yes', 'sunny', 'mild', 'normal', 'TRUE'],
                            ['yes', 'overcast', 'mild', 'high', 'TRUE'],
                            ['yes', 'overcast', 'hot', 'normal', 'FALSE'],
                            ['no', 'rainy', 'mild', 'high', 'TRUE']])

        X_train = dataset[:, 1:]
        y_train = dataset[:, 0]

        mae = MaxEntDFP(epsilon=1e-4, max_iter=1000, distance=0.01)
        # 训练模型
        mae.fit(X_train, y_train)
        print("模型训练迭代次数：{}次".format(mae.n_iter_))
        print("模型权重：{}".format(mae.w))

        result = mae.predict(['overcast', 'mild', 'high', 'FALSE'])
        print("预测结果：", result)
    else:
        print('running IIS/GIS')
        from sklearn.datasets import load_iris
        from sklearn.model_selection import train_test_split

        data = load_iris()
        X_data = data['data']
        y_data = data['target']
        X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.25, random_state=0)

        ME = MaxEnt(algorithms='')
        a = ME.fit(X_train, y_train)

        score = 0

        for X, y in zip(X_test, y_test):
            if ME.predict(X) == y:
                score += 1
        print(score / len(y_test) * 100)
        
        
if __name__ == '__main__':
    main('')



running IIS/GIS
this tool use a special way to get more eigenfunction
实际迭代次数200
81.57894736842105


### 测试

In [178]:
# IIS/GIS
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

data = load_iris()
X_data = data['data']
y_data = data['target']
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.25, random_state=0)

ME = MaxEnt(algorithms='')
a = ME.fit(X_train, y_train)

score = 0

for X, y in zip(X_test, y_test):
    if ME.predict(X) == y:
        score += 1
print(score / len(y_test) * 100)

this tool use a special way to get more eigenfunction
实际迭代次数200
81.57894736842105


In [179]:
# DFP
dataset = np.array([['no', 'sunny', 'hot', 'high', 'FALSE'],
                    ['no', 'sunny', 'hot', 'high', 'TRUE'],
                    ['yes', 'overcast', 'hot', 'high', 'FALSE'],
                    ['yes', 'rainy', 'mild', 'high', 'FALSE'],
                    ['yes', 'rainy', 'cool', 'normal', 'FALSE'],
                    ['no', 'rainy', 'cool', 'normal', 'TRUE'],
                    ['yes', 'overcast', 'cool', 'normal', 'TRUE'],
                    ['no', 'sunny', 'mild', 'high', 'FALSE'],
                    ['yes', 'sunny', 'cool', 'normal', 'FALSE'],
                    ['yes', 'rainy', 'mild', 'normal', 'FALSE'],
                    ['yes', 'sunny', 'mild', 'normal', 'TRUE'],
                    ['yes', 'overcast', 'mild', 'high', 'TRUE'],
                    ['yes', 'overcast', 'hot', 'normal', 'FALSE'],
                    ['no', 'rainy', 'mild', 'high', 'TRUE']])

X_train_new = dataset[:, 1:]
y_train_new = dataset[:, 0]

ME = MaxEnt(algorithms='')
ME.fit(X_train_new, y_train_new)

ME.predict(['overcast', 'mild', 'high', 'FALSE'])

this tool use a special way to get more eigenfunction
实际迭代次数200


'yes'

### 测试下来感觉IIS/GIS算法好用......